# Objectives
* download file from courses's cloud link
* get lateset exchange value from api
* run and test ETL process

# Set file path

In [2]:
target = 'bank_cap_exchange'
tmp_file = target + 'tmp.tmp'
log_file = target + '_log_file.txt'
target_file = target + '_transformed.csv'

### import library

In [14]:
from datetime import datetime
from urllib import request
import requests
import pandas as pd
import json
import glob

# Logging cell

In [3]:
def log(prioity,messenage):
    datetime_stamp = "%Y-%m-%d %H:%M:%S"
    now_time = datetime.now()
    datetime_log = now_time.strftime(datetime_stamp)
    with open(log_file,'a') as f:
        f.write(datetime_log + ',' + '[' + prioity + ']' + messenage + '\n')

In [81]:
log("action","Started to coding load function")

# Get data

In [6]:
url1= 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json'  
url2= 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json'
url3= 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv'
url_list = [url1,url2,url3]

In [7]:
#download all files into download folder
for url in url_list:
    response = request.urlretrieve(url,"download/" + url[url.rfind('/')+1:len(url)])

# ETL process

    #Extract function

In [22]:
#add folder path to use this function
def extract_from_json(file_to_load):
    dataframe = pd.read_json(file_to_load)
    return dataframe

        #Extract all files

In [49]:
def extract():
    extracted_data = pd.DataFrame(columns=[])
    for json_file in glob.glob('download/*.json'):
        extracted_data = extracted_data.append(extract_from_json(json_file),ignore_index=True)
    return extracted_data

    #Transform function

In [68]:
##api key
my_api_key = "NLyy5zY60LW5XcRteDkle9MmjyFZVs3a"
currency = 'GBP'

In [53]:
##exchange cal from api
def exchange_usd_set_index(my_api_key, currency):
    #get api data
    url = "https://api.apilayer.com/exchangerates_data/latest?base=EUR&apikey="+ my_api_key
    api_data = requests.get(url).text
    #clean data
    non_space_data = api_data.replace(' ','').replace('\n','')
    json_formed_data = json.loads(non_space_data)
    #input dataframe
    dataframe = pd.DataFrame(json_formed_data['rates'].items(),columns=['foreign_currency','rates'])
    dataframe_ind = dataframe.set_index('foreign_currency')
    
    return dataframe_ind.loc[currency]['rates']

In [ ]:
##transform function

In [74]:
def transform(data,curreny):
    #get rates
    ex = exchange_usd_set_index(my_api_key,currency)
    
    #exchange values
    data.iloc[:,1] = round(data.iloc[:,1] * ex,3)
    
    #rename column name
    replaced_name = data.columns[1].replace('US','GBP')
    data.rename(columns={data.columns[1]:replaced_name},inplace=True)
    
    return data

    #Load function

In [82]:
def load(data):
    data.to_csv(target_file)

# Running ETL process

In [85]:
log("action","Extract phase Started")
extracted_data = extract()
log("action","Extract phase Ended")
extracted_data

<ipython-input-49-26929d7b9358>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_json(json_file),ignore_index=True)
<ipython-input-49-26929d7b9358>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_json(json_file),ignore_index=True)


,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399
...,...,...
100,Ping An Bank,37.993
101,Standard Chartered,37.319
102,United Overseas Bank,35.128
103,QNB Group,33.560


In [87]:
log("action","Transform phase Started")
transformed_data = transform(extracted_data,currency)
log("action","Transform phase Ended")
transformed_data

,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,340.677
1,Industrial and Commercial Bank of China,300.835
2,Bank of America,283.508
3,Wells Fargo,268.416
4,China Construction Bank,224.309
...,...,...
100,Ping An Bank,33.109
101,Standard Chartered,32.521
102,United Overseas Bank,30.612
103,QNB Group,29.246


In [88]:
log("action","Load phase Started")
load(transformed_data)
log("action","Load phase Ended")

In [89]:
log("action","ETL process is complete")

In [90]:
log("action","check target file")

# Testing cell